In [1]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = user_secrets.get_secret("HUGGINGFACEHUB_API_TOKEN")
os.environ['WANDB_API_KEY'] = user_secrets.get_secret("WANDB_API_KEY ")

from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACEHUB_API_TOKEN")
login(token = hf_token)

In [2]:
!git clone https://github.com/namwasinyourheart/ft-llm/

Cloning into 'ft-llm'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 131 (delta 71), reused 96 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (131/131), 71.80 KiB | 4.49 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [3]:
cd ft-llm

/kaggle/working/ft-llm


In [4]:
!pip install -r requirements.txt -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
   ━━━

# GENERATION

In [84]:
%%writefile generate.py
from omegaconf import OmegaConf
from typing import List, Tuple
import importlib

from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    AutoConfig, 
    AutoModel,
)

from transformers import BitsAndBytesConfig
from src.utils.model_utils import set_torch_dtype_and_attn_implementation, get_quantization_config

def get_model_class(pretrained_model_name_or_path):
    """
    Dynamically retrieves the model class from Hugging Face transformers 
    using the model's configuration.

    Args:
        pretrained_model_name_or_path (str): The name or local path of the model.

    Returns:
        model_class (transformers.PreTrainedModel): The corresponding model class.
    """ 
    # Load the model configuration
    config = AutoConfig.from_pretrained(pretrained_model_name_or_path)

    # Extract the model class name
    model_class_name = config.architectures[0] if config.architectures else None

    if model_class_name:
        try:
            # Dynamically import the model class
            module = importlib.import_module("transformers")
            return getattr(module, model_class_name, AutoModel)
        except (ImportError, AttributeError):
            pass  # Fallback if import fails

    return AutoModel  # Default to AutoModel if class is not found

def get_quantization_config(model_args                          
) -> BitsAndBytesConfig | None:
    if model_args['load_in_4bit']:
        torch_dtype, attn_implementation = set_torch_dtype_and_attn_implementation()
        if model_args['bnb_4bit_compute_dtype']:
            torch_dtype = model_args['bnb_4bit_compute_dtype']

        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch_dtype,
            bnb_4bit_quant_type=model_args['bnb_4bit_quant_type'],
            bnb_4bit_use_double_quant=model_args['bnb_4bit_use_double_quant'],
            bnb_4bit_quant_storage=model_args['bnb_4bit_quant_storage'],
        ).to_dict()
    elif model_args['load_in_8bit']:
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True,
        ).to_dict()
    else:
        quantization_config = None

    return quantization_config


def load_model_for_generate(model_args, device_args):

    # Set torch_dtype and attn_implementation
    torch_dtype, attn_implementation = set_torch_dtype_and_attn_implementation()

    if model_args['torch_dtype']:
        torch_dtype = model_args['torch_dtype']

    # QLora Config
    quantization_config = get_quantization_config(model_args)

    model_class = get_model_class(model_args['pretrained_model_name_or_path'])

    # Load model
    model = model_class.from_pretrained(
            model_args['pretrained_model_name_or_path'],
            trust_remote_code=True,
            torch_dtype=torch_dtype,
            quantization_config=quantization_config if not device_args['use_cpu'] else None,
            device_map=model_args['device_map'],
            # device_map="cpu" if device_args['use_cpu'] else "auto",
            attn_implementation=attn_implementation,
            low_cpu_mem_usage=model_args['low_cpu_mem_usage']  # low_cpu_mem_usage=True if not device_args['use_cpu'] else False
    )
    
    return model

def load_model_vllm(model_args, device_args):
    # Set torch_dtype and attn_implementation
    torch_dtype, attn_implementation = set_torch_dtype_and_attn_implementation()

    # Load model
    llm = LLM(
        model=model_args['pretrained_model_name_or_path'],
        dtype=torch_dtype
        )

    return llm


def load_tokenizer_for_generate(
    model_args
) -> PreTrainedTokenizer:
    
    tokenizer = AutoTokenizer.from_pretrained(model_args['pretrained_tokenizer_name_or_path'])

    return tokenizer

def prepare_prompt(prompt_args):
    if prompt_args['use_no_keys']:
        return prompt_args['input']['text']

    intro = f"{prompt_args['intro_text']}"
    instruction = f"{prompt_args['instruction']['key']}\n{prompt_args['instruction']['text']}"

    examples = None
    if prompt_args['use_examples'] and 'examples' in prompt_args:
        example_template = prompt_args['examples']['template']
        formatted_examples = "\n\n".join(
            example_template.format(**example) for example in prompt_args['examples']['list']
        )
        examples = f"{prompt_args['examples']['key']}\n{formatted_examples}"

    input = f"{prompt_args['input']['key']}\n{prompt_args['input']['text']}"

    context = None
    if prompt_args['use_context'] and prompt_args['context']['key']:
        context = f"{prompt_args['context']['key']}\n{prompt_args['context']['text']}"

    response_key = f"{prompt_args['response_key']}\n"

    # Collect all non-empty parts and join them
    parts = [part for part in [intro, instruction, examples, context, input, response_key] if part]

    return "\n\n".join(parts)

def postprocess(prompt: str, tokenizer, output: str, response_key: str, end_key: str, return_full_text: bool=False):
    
    import re
    decoded = None
    fully_decoded = output
    
    if not end_key:
        end_key = tokenizer.eos_token
    
    pattern = r".*?{}\s*(.+?){}".format(response_key, end_key)
    m = re.search(pattern, fully_decoded, flags=re.DOTALL)

    if m:
        decoded = m.group(1).strip()
    else:
        # The model might not generate the end_key sequence before reaching the max tokens. In this case,
        # return everything after response_key.
        pattern = r".*?{}\s*(.+)".format(response_key)
        m = re.search(pattern, fully_decoded, flags=re.DOTALL)
        if m:
            decoded = m.group(1).strip()
        else:
            print(f"Failed to find response in:\n{fully_decoded}")
            
    if return_full_text:
        decoded = f"{prompt}{decoded}"

    return decoded


def generate_response(model_args, prompt_args, gen_args, device_args):

    from accelerate import Accelerator
    accelerator = Accelerator(cpu=device_args.use_cpu)
    
    model = load_model_for_generate(model_args, device_args)
    
    tokenizer = load_tokenizer_for_generate(model_args)

    prompt = prepare_prompt(prompt_args)

    input = tokenizer(prompt, return_tensors='pt', padding=False, truncation=True)
    input_ids = input.input_ids

    model = model.to(accelerator.device)
    input_ids = input_ids.to(accelerator.device)

    output_ids = model.generate(input_ids,
                                max_new_tokens=gen_args['max_new_tokens'], 
                                pad_token_id=tokenizer.pad_token_id)
    
    output = tokenizer.decode(output_ids[0], 
                              skip_special_tokens=True)

    if gen_args['do_postprocess']:
        output = postprocess(prompt, tokenizer, output, 
                             prompt_args['response_key'],
                             prompt_args['end_key'], 
                             gen_args['return_full'])
    
    return output


def parse_args():
    import argparse
    parser = argparse.ArgumentParser(description="Load generation config.")
    parser.add_argument("--config_path", type=str, required=True, help="Path to the YAML config file for generating.")
    parser.add_argument("--input_text", type=str, default=None, help="The text input (question).")

    return parser.parse_args()

def main():
    args = parse_args()

    # Load the generation config file
    cfg = OmegaConf.load(args.config_path)

    model_args = cfg.model
    prompt_args = cfg.prompt
    gen_args = cfg.generate
    device_args = cfg.device

    if args.input_text:
        prompt_args['input']['text'] = args.input_text

    response = generate_response(model_args, prompt_args, gen_args, device_args)
    print(response)

if __name__ == "__main__":
    main()
    

Overwriting generate.py


In [86]:
%%writefile configs/generate.yaml
model:
  pretrained_model_name_or_path: Qwen/Qwen2.5-7B-Instruct
  pretrained_tokenizer_name_or_path: Qwen/Qwen2.5-7B-Instruct
  load_in_4bit: true
  load_in_8bit: false
  bnb_4bit_compute_dtype: null
  bnb_4bit_quant_type: "nf4"
  bnb_4bit_use_double_quant: false
  bnb_4bit_quant_storage: "uint8"
  torch_dtype: float32
  attn_implementation: null
  device_map: null
  low_cpu_mem_usage: null
  lora_adapter: # path to lora adapter

prompt:
    use_no_keys: false
    use_examples: false
    use_context: false
    intro_text: "You are an expert in summarizing dialogue."
    instruction:
        key: "### Instruction:"
        text: "Summarize the following dialogue:"
    input:
        key: "### Dialogue:"
        text: |
            """Hannah: Hey, do you have Betty's number?
            Amanda: Lemme check
            Hannah: <file_gif>
            Amanda: Sorry, can't find it.
            Amanda: Ask Larry
            Amanda: He called her last time we were at the park together
            Hannah: I don't know him well
            Hannah: <file_gif>
            Amanda: Don't be shy, he's very nice
            Hannah: If you say so..
            Hannah: I'd rather you texted him
            Amanda: Just text him 🙂
            Hannah: Urgh.. Alright
            Hannah: Bye
            Amanda: Bye bye"""

    response_key: "### Summary:"
    end_key: null


generate:
  use_vllm: False
  do_postprocess: false
  return_full: false
  skip_special_tokens: true
  max_new_tokens: 512
  temperature: 0
  do_sample: null
  top_p: null
  # num_beams: 4,    
  # num_return_sequences: 4,
  # return_dict_in_generate: True,
  # output_scores: True,
  

device:
  use_cpu: false


Overwriting configs/generate.yaml


In [87]:
!python generate.py --config_path=configs/generate.yaml

2025-02-24 17:35:39.265411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-24 17:35:39.467182: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-24 17:35:39.526424: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████████████| 4/4 [01:09<00:00, 17.47s/